In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table_body = soup.find('tbody')

column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
data_frame = pd.DataFrame(columns=column_names)

for single_row in table_body.find_all('tr'):
    cols = single_row.find_all('td')
    if len(cols) > 0:
        postal_code = cols[0].text
        borough = cols[1].text
        neighborhood = cols[2].text.replace('\n','')
        
        if not borough == 'Not assigned': 
            if neighborhood == 'Not assigned':
                neighborhood = borough
            
            is_repeated = False
            
            for index, row in data_frame.iterrows():
                if row[0] == postal_code:
                    is_repeated = True
                    data_frame.loc[index]['Neighborhood'] = data_frame.loc[index]['Neighborhood'] + ', ' + neighborhood
            if not is_repeated:
                data_frame = data_frame.append({'PostalCode': postal_code,
                                                'Borough': borough,
                                                'Neighborhood': neighborhood}, ignore_index=True)

<H2> Geocoder not returning values <H2>

In [ ]:
import geocoder

for index, row in data_frame.iterrows():
    postal_code = row['PostalCode']
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    data_frame.loc[index]['Latitude'] = latitude
    data_frame.loc[index]['Longitude'] = longitude


<H2> Retrieving LL Values from Text File Provided </H2>

In [36]:
pd.options.mode.chained_assignment = None 
ll_data = pd.read_csv("Geospatial_Coordinates.csv")

for index, row in data_frame.iterrows():
    postal_code = row['PostalCode']
    ll_row = ll_data.loc[ll_data['Postal Code'] == postal_code]

    data_frame.loc[index]['Latitude'] = float(ll_row['Latitude'])
    data_frame.loc[index]['Longitude'] = float(ll_row['Longitude'])
    
    
print(data_frame.head())


  PostalCode           Borough                      Neighborhood Latitude  \
0        M3A        North York                         Parkwoods  43.7533   
1        M4A        North York                  Victoria Village  43.7259   
2        M5A  Downtown Toronto         Harbourfront, Regent Park  43.6543   
3        M6A        North York  Lawrence Heights, Lawrence Manor  43.7185   
4        M7A      Queen's Park                      Queen's Park  43.6623   

  Longitude  
0  -79.3297  
1  -79.3156  
2  -79.3606  
3  -79.4648  
4  -79.3895  
